In [1]:
# Some useful functions, stolen from Peter Norvig: https://github.com/norvig/pytudes/blob/master/ipynb/Advent%202017.ipynb

def Input(day):
    "Open this day's input file."
    filename = 'data/input{}.txt'.format(day)
    return open(filename)


def Inputstr(day): 
    "The contents of this day's input file as a str."
    return Input(day).read().rstrip('\n')


def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def Vector(line):
    "Parse a str into a tuple of atoms (numbers or str tokens)."
    return mapt(Atom, line.replace(',', ' ').split())

def Atom(token):
    "Parse a str token into a number, or leave it as a str."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

# Day 1

In [2]:
def fuel(mass):
    return (mass // 3) - 2
    
masses = mapt(int, Input(1))
N = len(masses)
N

100

### Part 1

In [3]:
part1 = 0
for mass in masses:
    part1 += fuel(mass)

assert part1 == 3154112
part1

3154112

### Part 2

In [4]:
def fuelSq(mass):
    f = fuel(mass)
    return f + fuelSq(f) if f > 0 else 0

part2 = 0
for mass in masses:
    part2 += fuelSq(mass)
assert part2 == 4728317
part2

4728317

# Day 2

In [5]:
base = [int(x) for x in Input(2).read().split(',')]

def reset():
    return [x for x in base]
    
prog = reset()
N = len(prog)
N

145

### Part 1

In [6]:
def run(noun, verb):
    prog = reset()
    
    prog[1] = noun
    prog[2] = verb

    pos = 0
    while True:
        inst = prog[pos]
        i1,i2,out = prog[pos+1],prog[pos+2],prog[pos+3]
        if inst == 99:
            break
        elif inst == 1:
            prog[out] = prog[i1] + prog[i2]
        elif inst == 2:
            prog[out] = prog[i1] * prog[i2]
        else:
            raise Exception("Bad instruction: {}".format(inst)) 
        pos = pos + 4
    
    return prog[0]
        

part1 = run(12, 2)
assert part1 == 3224742
part1

3224742

### Part 2

In [7]:
def go():
    for noun in range(0, 100):
        for verb in range(0, 100):
            try:
                result = run(noun, verb)
                if result == 19690720:
                    print("Found it: {}, {}".format(noun, verb))
                    return (100 * noun) + verb
            except:
                print("{}, {} errors".format(noun, verb))
                
part2 = go()
assert part2 == 7960
part2

Found it: 79, 60


7960

# Day 3

In [46]:
def X(point): return point[0]
def Y(point): return point[1]

origin = (0, 0)
HEADINGS = { "U": (0, -1), "L": (-1, 0), "D": (0, 1), "R": (1, 0) }

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def segment(start, directionStr, distance):
    direction = HEADINGS[directionStr]
    vect = (direction[0] * distance, direction[1] * distance)
    end = add(start, vect)
    return (start, end)

def ccw(A,B,C):
    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

# Return true if line segments AB and CD intersect
def intersect(A,B,C,D):
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

def intersectPt(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(line1[0],line1[1]), det(line2[0],line2[1]))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return int(x), int(y)

In [9]:
file = Input(3)
w1 = [str(x) for x in file.readline().split(',')]
w2 = [str(x) for x in file.readline().split(',')]

(len(w1),len(w2))

(301, 301)

### Part 1

In [47]:
w1p = []
for s in w1:
    directionStr = s[:1]
    distance = int(s[1:])
    start = origin if len(w1p) == 0 else w1p[-1][1]
    w1p.append(segment(start, directionStr, distance))

w2p = []
for s in w2:
    directionStr = s[:1]
    distance = int(s[1:])
    start = origin if len(w2p) == 0 else w2p[-1][1]
    w2p.append(segment(start, directionStr, distance))

cross = []
for i in w1p:
    for j in w2p:
        if intersect(i[0],i[1],j[0],j[1]):
            cross.append(intersectPt(i,j))

shortest = 99999999999999
for pt in cross:
    dist = cityblock_distance(origin, pt)
    if dist < shortest:
        shortest = dist

assert shortest == 209
shortest

209

### Part 2

In [49]:
w1pb = []
for s in w1:
    directionStr = s[:1]
    distance = int(s[1:])
    start = origin if len(w1pb) == 0 else w1pb[-1][1]
    line = segment(start, directionStr, distance)
    w1pb.append((line[0],line[1],distance))

w2pb = []
for s in w2:
    directionStr = s[:1]
    distance = int(s[1:])
    start = origin if len(w2pb) == 0 else w2pb[-1][1]
    line = segment(start, directionStr, distance)
    w2pb.append((line[0],line[1],distance))

cross = []
for i in w1pb:
    for j in w2pb:
        if intersect(i[0],i[1],j[0],j[1]):
            cross.append(intersectPt(i,j))

shortest = 99999999999999
for pt in cross:
    dist = cityblock_distance(origin, pt)
    if dist < shortest:
        shortest = dist

shortest

209